### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import Audio
import torch.optim as optim
from types import SimpleNamespace
import scipy.signal as sc
import time
from sklearn.metrics import f1_score
import pickle

from trainDataset import TrainDataset
from testDataset import TestDataset
#from trainDatasetNew import TrainDatasetNew
#from testDatasetNew import TestDatasetNew
from validation_split import get_dataloaders
from math_utils import logMagStft, ffts
from SpectrogramCNN import SpectrogramCNN
from train_utils import train, test, save_output, save_geometric_mean_predictions
from evaluation_utils import get_mean_F1, get_loss
from MulitScale1DResNet import MSResNet
from LSTM import LSTM
from FFTMulitScale1DResNet import FFTMSResNet
from SpectralResNet import SpectralResNet18, SpectralResNet34
from scipy.signal import hilbert
from FeatureFNN import FeatureFNN
from MelRawCombined import CNN1D, CNN2D

### Parameters

In [27]:
validation_split = .1
do_plots = False
load_model = True
args = SimpleNamespace(batch_size=4, test_batch_size=4, epochs=30,
                       lr=0.0001, momentum=0.9, seed=1, log_interval=200, weight_decay=0.0001,
                      net = SpectralResNet34) #SpectrogramCNN, MSResNet, SpectralResNet34, FFTMSResNet, FeatureFNN, CNN1D, CNN2D 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.cuda.is_available(): # adapt those paths on other machine
    print('no cuda')
    path_train = './../data/train-small/'
    path_test =  './../data/test/kaggle-test/'
else:
    print('with cuda')
    path_train = './../data/kaggle-train/'
    path_test =  './../data/kaggle-test/'
    
path_model = 'models/'
path_submission = 'submissions/'
    
sample_rate = 16000
nmbr_classes = 10

with cuda


### Original Dataset

In [3]:
# todo add in the classes the features and the fft data

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset(path_train, transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset(path_test, transform=toFloat)
print(len(testDataset))

269776
4096


In [4]:
input_size = len(trainDataset[0][0])
print('input size: ',input_size)

input size:  64000


### Look at Original Data

In [5]:
if do_plots:
    # how many instruments are there?
    dummy_count = np.zeros(20)

    for sample in trainDataset:
        dummy_count[sample[1]] += 1

    labels_count = []
    for elem in dummy_count:
        if elem != 0:
            labels_count.append(elem)

    print(labels_count)

In [6]:
if do_plots:
    nmbr_classes = len(labels_count)
    print('nmbr_classes: ', nmbr_classes)

In [7]:
if do_plots:
    plt.plot(labels_count, '*')

In [8]:
if do_plots:
    # plot one of each

    done = np.zeros(nmbr_classes)
    examples = []

    for sample in trainDataset:
        if done[sample[1]] == 0:
            examples.append(sample)
            done[sample[1]] = 1

In [9]:
plt.rcParams["figure.figsize"] = (20,10)

In [10]:
#time series

if do_plots:

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        plt.plot(examples[i][0])

In [11]:
#envelope

if do_plots:
    
    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        env = torch.from_numpy(np.abs(hilbert(examples[i][0]))).view(1,1,-1)
        env = m(env)
        plt.plot(env[0,0].numpy())
        
    plt.show()

In [12]:
# fft

if do_plots:

    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        fft = torch.from_numpy(ffts(examples[i][0])).view(1,1,-1)
        fft = m(fft)
        plt.plot(fft[0,0].numpy())
        
    plt.show()

In [13]:
# melspec

if do_plots:
    n_fft = 510
    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        spectrogram = logMagStft(examples[i][0], 16000, n_fft)
        plt.imshow(spectrogram)
        
    plt.show()

In [14]:
if do_plots:

    for sample in examples:
        display(Audio(sample[0], rate=sample_rate))

### Dataloaders

In [15]:
# validation split is done here

train_loader, validation_loader, indices = get_dataloaders(trainDataset, 
                                                  batch_size = args.batch_size, 
                                                  validation_split = validation_split, 
                                                  shuffle_dataset = True, 
                                                  random_seed = None)#,
                                                  #indices = indices[0])

for samples, instrument_family_target in train_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break
        
for samples, instrument_family_target in validation_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break

torch.Size([4, 64000]) torch.Size([4]) tensor([3, 4, 5, 0])
tensor(-0.9380, dtype=torch.float64) tensor(0.9464, dtype=torch.float64)
['guitar' 'keyboard' 'mallet' 'bass']
torch.Size([4, 64000]) torch.Size([4]) tensor([2, 8, 9, 0])
tensor(-0.9466, dtype=torch.float64) tensor(0.9466, dtype=torch.float64)
['flute' 'string' 'vocal' 'bass']


In [26]:
# save indices
with open(path_submission + "indices.txt", "wb") as fp:
    pickle.dump([indices], fp)

In [17]:
test_loader = data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False) #!!! shuffle should be false
for samples in test_loader:
        print(samples[0].shape)
        print(torch.min(samples[0]), torch.max(samples[0]))
        break

torch.Size([4, 64000])
tensor(-0.8568, dtype=torch.float64) tensor(0.9467, dtype=torch.float64)


### Main

In [18]:
if (args.net == SpectrogramCNN) or (args.net == SpectralResNet18) \
    or (args.net == FeatureFNN) or (args.net == CNN1D) \
    or (args.net == CNN2D) or (args.net == SpectralResNet34):
    model = args.net(device).to(device)

In [19]:
if (args.net == MSResNet) or (args.net == FFTMSResNet):
    model = args.net(1, device).to(device)

In [20]:
if args.net == LSTM:
    model = args.net(device, input_size = 252, hidden_size = 512, num_layers = 1, num_classes = 10).to(device)

In [21]:
print(model)

SpectralResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1,

In [22]:
# Main
#optimizer = optim.Adam(model.parameters(), lr=args.lr)

optimizer = optim.SGD(model.parameters(), lr= args.lr, momentum= args.momentum, weight_decay= args.weight_decay)

info = {'highest F1' : 0,
        'lowest loss' : 100,
        'saved epoch' : 10}

In [31]:
for epoch in range(1 +7, args.epochs + 1 +7):
    
    #if epoch%7 == 0:
    #    for param_group in optimizer.param_groups:
    #        param_group['lr'] = param_group['lr'] / 10
    #        print('New Learning Rate: ', param_group['lr'])
    
    train(args, model, device, train_loader, optimizer, epoch, start_time = time.time())
    #f1 = get_mean_F1(model, validation_loader)
    loss_epoch, mean_f1 = get_loss(model, validation_loader, device)
    #print('after epoch {} got f1 score of {}'.format(epoch , f1))
    print('after epoch {} got loss of {} and f1 of {}'.format(epoch , loss_epoch, mean_f1))
    #if f1 > info['highest F1']:
    #    info['highest F1'] = np.copy(f1)
    #    info['saved epoch'] = epoch 
    #    test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
    #    torch.save(model, path_model)
    #    print('currently best model --> saved')
    if loss_epoch < info['lowest loss']:
        info['lowest loss'] = np.copy(loss_epoch)
        info['highest F1'] = np.copy(mean_f1)
        info['saved epoch'] = epoch 
        test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
        torch.save(model, path_model + 'model_' + str(epoch) + 'epoch.pt')
        print('currently best model --> saved')    
        
print(info)

Train Epoch: 8 [0/269776 (0%)]	Loss: 0.000048	F1: 1.0000	Runtime: 0.2
Train Epoch: 8 [800/269776 (0%)]	Loss: 0.000707	F1: 1.0000	Runtime: 35.4
Train Epoch: 8 [1600/269776 (1%)]	Loss: 0.000004	F1: 1.0000	Runtime: 70.6
Train Epoch: 8 [2400/269776 (1%)]	Loss: 0.000020	F1: 1.0000	Runtime: 105.9
Train Epoch: 8 [3200/269776 (1%)]	Loss: 0.001192	F1: 1.0000	Runtime: 141.2
Train Epoch: 8 [4000/269776 (2%)]	Loss: 0.000005	F1: 1.0000	Runtime: 176.7
Train Epoch: 8 [4800/269776 (2%)]	Loss: 0.020992	F1: 1.0000	Runtime: 212.3
Train Epoch: 8 [5600/269776 (2%)]	Loss: 0.000195	F1: 1.0000	Runtime: 247.7
Train Epoch: 8 [6400/269776 (3%)]	Loss: 0.181952	F1: 0.7500	Runtime: 283.2
Train Epoch: 8 [7200/269776 (3%)]	Loss: 0.000000	F1: 1.0000	Runtime: 318.8
Train Epoch: 8 [8000/269776 (3%)]	Loss: 0.000037	F1: 1.0000	Runtime: 354.3
Train Epoch: 8 [8800/269776 (4%)]	Loss: 0.000092	F1: 1.0000	Runtime: 389.7
Train Epoch: 8 [9600/269776 (4%)]	Loss: 0.000064	F1: 1.0000	Runtime: 425.2
Train Epoch: 8 [10400/269776 (4%)

Train Epoch: 8 [85600/269776 (35%)]	Loss: 0.000003	F1: 1.0000	Runtime: 3787.5
Train Epoch: 8 [86400/269776 (36%)]	Loss: 0.000004	F1: 1.0000	Runtime: 3822.9
Train Epoch: 8 [87200/269776 (36%)]	Loss: 0.000993	F1: 1.0000	Runtime: 3858.2
Train Epoch: 8 [88000/269776 (36%)]	Loss: 0.001828	F1: 1.0000	Runtime: 3893.6
Train Epoch: 8 [88800/269776 (37%)]	Loss: 0.000002	F1: 1.0000	Runtime: 3929.0
Train Epoch: 8 [89600/269776 (37%)]	Loss: 0.000530	F1: 1.0000	Runtime: 3964.4
Train Epoch: 8 [90400/269776 (37%)]	Loss: 0.000001	F1: 1.0000	Runtime: 3999.7
Train Epoch: 8 [91200/269776 (38%)]	Loss: 0.000035	F1: 1.0000	Runtime: 4035.1
Train Epoch: 8 [92000/269776 (38%)]	Loss: 0.000000	F1: 1.0000	Runtime: 4070.4
Train Epoch: 8 [92800/269776 (38%)]	Loss: 0.000101	F1: 1.0000	Runtime: 4105.8
Train Epoch: 8 [93600/269776 (39%)]	Loss: 0.000001	F1: 1.0000	Runtime: 4141.2
Train Epoch: 8 [94400/269776 (39%)]	Loss: 0.000010	F1: 1.0000	Runtime: 4176.6
Train Epoch: 8 [95200/269776 (39%)]	Loss: 0.000107	F1: 1.0000	Ru

Train Epoch: 8 [168800/269776 (70%)]	Loss: 0.000010	F1: 1.0000	Runtime: 7465.8
Train Epoch: 8 [169600/269776 (70%)]	Loss: 0.000000	F1: 1.0000	Runtime: 7501.2
Train Epoch: 8 [170400/269776 (70%)]	Loss: 0.000077	F1: 1.0000	Runtime: 7536.6
Train Epoch: 8 [171200/269776 (71%)]	Loss: 0.000080	F1: 1.0000	Runtime: 7571.9
Train Epoch: 8 [172000/269776 (71%)]	Loss: 0.000017	F1: 1.0000	Runtime: 7608.5
Train Epoch: 8 [172800/269776 (71%)]	Loss: 0.000005	F1: 1.0000	Runtime: 7644.8
Train Epoch: 8 [173600/269776 (71%)]	Loss: 0.000046	F1: 1.0000	Runtime: 7680.3
Train Epoch: 8 [174400/269776 (72%)]	Loss: 0.000141	F1: 1.0000	Runtime: 7715.9
Train Epoch: 8 [175200/269776 (72%)]	Loss: 0.001842	F1: 1.0000	Runtime: 7751.3
Train Epoch: 8 [176000/269776 (72%)]	Loss: 0.000035	F1: 1.0000	Runtime: 7786.7
Train Epoch: 8 [176800/269776 (73%)]	Loss: 0.000004	F1: 1.0000	Runtime: 7822.1
Train Epoch: 8 [177600/269776 (73%)]	Loss: 0.000000	F1: 1.0000	Runtime: 7857.5
Train Epoch: 8 [178400/269776 (73%)]	Loss: 0.000473	

Train Epoch: 9 [8000/269776 (3%)]	Loss: 0.000128	F1: 1.0000	Runtime: 354.0
Train Epoch: 9 [8800/269776 (4%)]	Loss: 0.000016	F1: 1.0000	Runtime: 389.3
Train Epoch: 9 [9600/269776 (4%)]	Loss: 0.000001	F1: 1.0000	Runtime: 424.7
Train Epoch: 9 [10400/269776 (4%)]	Loss: 0.000004	F1: 1.0000	Runtime: 460.1
Train Epoch: 9 [11200/269776 (5%)]	Loss: 0.000023	F1: 1.0000	Runtime: 495.5
Train Epoch: 9 [12000/269776 (5%)]	Loss: 0.000858	F1: 1.0000	Runtime: 530.8
Train Epoch: 9 [12800/269776 (5%)]	Loss: 0.000003	F1: 1.0000	Runtime: 566.2
Train Epoch: 9 [13600/269776 (6%)]	Loss: 0.000126	F1: 1.0000	Runtime: 601.6
Train Epoch: 9 [14400/269776 (6%)]	Loss: 0.000009	F1: 1.0000	Runtime: 637.0
Train Epoch: 9 [15200/269776 (6%)]	Loss: 0.000110	F1: 1.0000	Runtime: 672.3
Train Epoch: 9 [16000/269776 (7%)]	Loss: 0.000174	F1: 1.0000	Runtime: 707.7
Train Epoch: 9 [16800/269776 (7%)]	Loss: 0.000000	F1: 1.0000	Runtime: 743.1
Train Epoch: 9 [17600/269776 (7%)]	Loss: 0.000096	F1: 1.0000	Runtime: 778.5
Train Epoch: 9 

Train Epoch: 9 [92800/269776 (38%)]	Loss: 0.000016	F1: 1.0000	Runtime: 4104.0
Train Epoch: 9 [93600/269776 (39%)]	Loss: 0.000022	F1: 1.0000	Runtime: 4139.4
Train Epoch: 9 [94400/269776 (39%)]	Loss: 0.000000	F1: 1.0000	Runtime: 4174.8
Train Epoch: 9 [95200/269776 (39%)]	Loss: 0.000001	F1: 1.0000	Runtime: 4210.1
Train Epoch: 9 [96000/269776 (40%)]	Loss: 0.000021	F1: 1.0000	Runtime: 4245.5
Train Epoch: 9 [96800/269776 (40%)]	Loss: 0.000041	F1: 1.0000	Runtime: 4280.9
Train Epoch: 9 [97600/269776 (40%)]	Loss: 0.000022	F1: 1.0000	Runtime: 4316.3
Train Epoch: 9 [98400/269776 (41%)]	Loss: 0.000001	F1: 1.0000	Runtime: 4351.6
Train Epoch: 9 [99200/269776 (41%)]	Loss: 0.000000	F1: 1.0000	Runtime: 4387.0
Train Epoch: 9 [100000/269776 (41%)]	Loss: 0.000001	F1: 1.0000	Runtime: 4422.4
Train Epoch: 9 [100800/269776 (42%)]	Loss: 0.000023	F1: 1.0000	Runtime: 4457.8
Train Epoch: 9 [101600/269776 (42%)]	Loss: 0.000028	F1: 1.0000	Runtime: 4493.2
Train Epoch: 9 [102400/269776 (42%)]	Loss: 0.000023	F1: 1.000

Train Epoch: 9 [176000/269776 (72%)]	Loss: 0.000227	F1: 1.0000	Runtime: 7783.7
Train Epoch: 9 [176800/269776 (73%)]	Loss: 0.000021	F1: 1.0000	Runtime: 7819.0
Train Epoch: 9 [177600/269776 (73%)]	Loss: 0.000017	F1: 1.0000	Runtime: 7854.4
Train Epoch: 9 [178400/269776 (73%)]	Loss: 0.000000	F1: 1.0000	Runtime: 7889.8
Train Epoch: 9 [179200/269776 (74%)]	Loss: 0.000001	F1: 1.0000	Runtime: 7925.2
Train Epoch: 9 [180000/269776 (74%)]	Loss: 0.000004	F1: 1.0000	Runtime: 7960.5
Train Epoch: 9 [180800/269776 (74%)]	Loss: 0.000000	F1: 1.0000	Runtime: 7995.9
Train Epoch: 9 [181600/269776 (75%)]	Loss: 0.000010	F1: 1.0000	Runtime: 8031.3
Train Epoch: 9 [182400/269776 (75%)]	Loss: 0.008852	F1: 1.0000	Runtime: 8066.7
Train Epoch: 9 [183200/269776 (75%)]	Loss: 0.000000	F1: 1.0000	Runtime: 8102.1
Train Epoch: 9 [184000/269776 (76%)]	Loss: 0.000640	F1: 1.0000	Runtime: 8137.4
Train Epoch: 9 [184800/269776 (76%)]	Loss: 0.000007	F1: 1.0000	Runtime: 8172.8
Train Epoch: 9 [185600/269776 (76%)]	Loss: 0.000003	

Train Epoch: 10 [15200/269776 (6%)]	Loss: 0.000078	F1: 1.0000	Runtime: 675.9
Train Epoch: 10 [16000/269776 (7%)]	Loss: 0.000000	F1: 1.0000	Runtime: 711.6


KeyboardInterrupt: 

### save or load outputs

In [34]:
save_output(args, model, device, test_loader, 'SpectralResNet34-score776', 
            trainDataset, testDataset, path_save = path_submission)

saved outputs


In [35]:
save_geometric_mean_predictions(path_submission + 'output-MSResNet-score701.txt', 
                                path_submission + 'output-SpectralResNet34-score776.txt', 
                                path_submission, 
                                trainDataset, testDataset)

saved predictions


### Load Model

In [33]:
if load_model:
    #model = torch.load(path_model)
    model = torch.load('models/SpectralResNet34_7epoch_score776.pt')

In [4]:
# load indices
with open(path_submission + "indices_MSResNet_score701.pt", "rb") as fp:
    indices = pickle.load(fp)

In [23]:
#get_mean_F1(model, validation_loader)
get_loss(model, validation_loader,device)

(2.308869604184242, 0.03617494440326128)

In [30]:
#epoch=10
#test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)

for param_group in optimizer.param_groups:
    param_group['lr'] = param_group['lr'] / 10
    print('New Learning Rate: ', param_group['lr'])

New Learning Rate:  1e-05
